In [ ]:
import sys
import os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
import math   # for mathematical operations
import random
from glob import glob
import cv2
from keras.utils import to_categorical

In [ ]:
num_features = 64
num_labels = 2
batch_size = 128
epochs = 1
width, height = 224, 224

In [ ]:
TRAIN_FOLDER_PATH = r'C:\\Users\djcoo\Documents\LYIT\Final\project\RWF-2000_test\train'
TEST_FOLDER_PATH = r'C:\\Users\djcoo\Documents\LYIT\Final\project\RWF-2000_test\val'

def listDir(dir):
    catgeory = {}
    dirNames = os.listdir(dir)
    for dirName in dirNames:
        catgeory[dirName] = []
#         print('Directory Name : '+ dirName)
        dir_path = os.path.abspath(os.path.join(dir, dirName))
#         print('Directory Path : '+ dir_path)
        fileNames = os.listdir(dir_path)
        for fileName in fileNames:
            file_path = os.path.abspath(os.path.join(dir_path, fileName))
#             print('File Path : '+ file_path )
            catgeory[dirName].append(file_path)
    return catgeory

LIST_OF_TRAIN_CATEGORY = listDir(TRAIN_FOLDER_PATH)
LIST_OF_TEST_CATEGORY = listDir(TEST_FOLDER_PATH)

In [ ]:
def get_video_frames(path_arr, flag):
    label_arr = ["Fight", "NonFight"]
    for label in label_arr:
        image_count = 0
        for video_path in tqdm(path_arr[label]):
#             video_name = video_path.split('\\').pop().split(".")[0]
        #     print(video_name)
            cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
            frameRate = cap.get(5) #frame rate
            frame_count = 0
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % math.floor(frameRate) == 0):
                    # storing the frames in a new folder named train_1
                    TRAIN_FRAME_PATH = "C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\"+ flag +"\\" + label + "_"
#                     filename = r"%s" %TRAIN_FRAME_PATH + video_name +"_frame%d.jpg" %frame_count;
                    filename = r"%s" %TRAIN_FRAME_PATH + "%d"%image_count +"_frame%d.jpg" %frame_count
#                     print(filename)
                    frame_count+=1
                    cv2.imwrite(filename, frame)
            image_count+=1
            cap.release()

get_video_frames(LIST_OF_TRAIN_CATEGORY, "train")
get_video_frames(LIST_OF_TEST_CATEGORY, "test")

In [ ]:
images = glob(r"C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\train\\*.jpg")
# creating an empty list
random.shuffle(images)

train_image = []
train_class = []

# for loop to read and store frames
for img_path in tqdm(images):
    # loading the image and keeping the target size as (224,224,3)
#     img = image.load_img(img_path, target_size=(224,224,3))
#     # converting it to array
#     img = image.img_to_array(img)
#     # normalizing the pixel value
#     img = img/255
#     # appending the image to the train_image list
#     print(img_path) 
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    train_image.append(image)
    train_class.append(img_path.split("\\").pop().split("_")[0])

In [ ]:
images = glob(r"C:\\Users\\djcoo\\Documents\\LYIT\\Final\\project\\frames\\test\\*.jpg")
# creating an empty list
random.shuffle(images)

test_image = []
test_class = []

# for loop to read and store frames
for img_path in tqdm(images):
#     # loading the image and keeping the target size as (224,224,3)
#     img = image.load_img(img_path, target_size=(224,224,3))
#     # converting it to array
#     img = image.img_to_array(img)
#     # normalizing the pixel value
#     img = img/255
    # appending the image to the train_image list
#     print(img_path) 
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = cv2.resize(image, (224, 224))
    
    test_image.append(image)
    test_class.append(img_path.split("\\").pop().split("_")[0])
    


In [ ]:
# lb = LabelBinarizer()
# y_train = lb.fit_transform(train_class)
# y_test = lb.fit_transform(test_class)

# print(len(y_train))
# print(len(y_test))

# converting the list to numpy array
X_train = np.array(train_image,'float32')
X_test = np.array(test_image,'float32')
#y_train = np.array(train_class,'float32')
# y_test = np.array(test_class,'float32')
y_train = to_categorical(lb.fit_transform(train_class))
y_test = to_categorical(lb.fit_transform(test_class))
print(y_test)

# train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
# test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

# #cannot produce
# #normalizing data between 0 and 1
# X_train -= np.median(X_train, axis=0)
# X_train /= np.std(X_train, axis=0)

# X_test -= np.median(X_test, axis=0)
# X_test /= np.std(X_test, axis=0)

In [ ]:
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))


model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True)


#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")